In [43]:
# !pip install geopy

In [57]:
import requests 
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [100]:
# Get exact latitude and longitude of the address

input_address = '285 Peyton Ct'
geolocator = Nominatim(user_agent="ish")
location = geolocator.geocode(input_address)

address = location.address
latitude = location.latitude
longitude = location.longitude


print("Address:", address)
print("Latitude:", latitude)
print("Longitude:", longitude)

Address: 285, Peyton Court, Copeley Hill Apartments, University of Virginia, Albemarle County, Virginia, 22903, United States
Latitude: 38.0480118
Longitude: -78.5091330774942


In [101]:
# Using the Overpass API to retrieve map data

overpass_url = "http://overpass-api.de/api/interpreter"

In [102]:
# Function to get grocery stores around a given latitude and longitude

def find_grocery_stores(latitude, longitude, radius=1.0):

    overpass_query = f"""
    [out:json];
    (
      node["shop"="supermarket"](around:{radius * 1000}, {latitude}, {longitude});
      node["shop"="grocery"](around:{radius * 1000}, {latitude}, {longitude});
    );
    out body;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        stores = data['elements']
        
        if not stores:
            print("No grocery stores found within the specified radius.")
        else:

            stores_with_distance = [] # to collect and sort by distance
            for store in stores:
                store_location = (store['lat'], store['lon'])
                store_name = store.get('tags', {}).get('name', 'Unnamed')
                distance = geodesic((latitude, longitude), store_location).km
                stores_with_distance.append((store_name, store_location, distance))
            
            stores_with_distance.sort(key=lambda x: x[2])

            print(f"Found {len(stores)} grocery stores within {radius} km, sorted by distance:")
            for store_name, store_location, distance in stores_with_distance:
                print(f"Name: {store_name}, Coordinates: {store_location}, Distance: {distance:.2f} km")
    else:
        print("Error fetching data from Overpass API. Please try again later.")

In [ ]:
# Function to find bus stops around a given latitude and longitude

def find_bus_stops(latitude, longitude, radius=1.0):
    overpass_query = f"""
    [out:json];
    node["highway"="bus_stop"](around:{radius * 1000}, {latitude}, {longitude});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        bus_stops = data['elements']
        
        if not bus_stops:
            print("No bus stops found within the specified radius.")
        else:
           
            bus_stops_with_distance = []  
            for stop in bus_stops:
                stop_location = (stop['lat'], stop['lon'])
                stop_name = stop.get('tags', {}).get('name', 'Unnamed')
                network_provider = stop.get('tags', {}).get('network', 'Unknown')
                distance = geodesic((latitude, longitude), stop_location).km
                bus_stops_with_distance.append((stop_name, network_provider, stop_location, distance))
        

            bus_stops_with_distance.sort(key=lambda x: x[2]) 

            print(f"Found {len(bus_stops)} bus stops within {radius} km, sorted by distance:")
            for stop_name, network_provider, stop_location, distance in bus_stops_with_distance:
                print(f"Name: {stop_name}, Network: {network_provider}, Coordinates: {stop_location}, Distance: {distance:.2f} km")
    else:
        print("Error fetching data from Overpass API. Please try again later.")

In [ ]:
# Function to get node details by ID - used for 'way' type elements 

def get_node_details(node_id):
    overpass_query = f"""
    [out:json];
    node({node_id});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    if response.status_code == 200:
        data = response.json()
        if 'elements' in data and data['elements']:
            return data['elements'][0]
    return None

In [134]:
# Function to get gyms around a given latitude and longitude

def find_gyms(latitude, longitude, radius=1.0):
    overpass_query = f"""
    [out:json];
    (
    node["leisure"="fitness_centre"](around:{radius * 1000}, {latitude}, {longitude});
    node["sport"="gym"](around:{radius * 1000}, {latitude}, {longitude});
    node["leisure"="recreation_ground"](around:{radius * 1000}, {latitude}, {longitude});
    way["leisure"="sports_centre"](around:{radius * 1000}, {latitude}, {longitude});
    relation["leisure"="sports_centre"](around:{radius * 1000}, {latitude}, {longitude});
    node["amenity"="gym"](around:{radius * 1000}, {latitude}, {longitude});
    node["sport"="fitness"](around:{radius * 1000}, {latitude}, {longitude});
    node["amenity"="recreation_centre"](around:{radius * 1000}, {latitude}, {longitude});
    node["leisure"="sports_centre"](around:{radius * 1000}, {latitude}, {longitude});
    );
    out body;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        elements = data['elements']
        
        if not elements:
            print("No sports centers found within the specified radius.")
        else:
            centres_with_distance = []

            for element in elements:
                if element['type'] == 'node':
                    location = (element['lat'], element['lon'])
                    name = element.get('tags', {}).get('name', 'Unnamed')
                    distance = geodesic((latitude, longitude), location).km
                    centres_with_distance.append((name, location, distance))
                
                elif element['type'] == 'way':
                    if 'nodes' in element and element['nodes']:
                        node_id = element['nodes'][0]  # get the first node of the way
                        node_data = next((item for item in elements if item['type'] == 'node' and item['id'] == node_id), None)
                        
                        if not node_data:  # fetch node details if not found in the initial response
                            node_data = get_node_details(node_id)
                        
                        if node_data:
                            location = (node_data['lat'], node_data['lon'])
                            name = element.get('tags', {}).get('name', 'Unnamed')
                            distance = geodesic((latitude, longitude), location).km
                            centres_with_distance.append((name, location, distance))

            centres_with_distance.sort(key=lambda x: x[2])

            print(f"Found {len(centres_with_distance)} sports centers within {radius} km, sorted by distance:")
            for name, location, distance in centres_with_distance:
                print(f"Name: {name}, Coordinates: {location}, Distance: {distance:.2f} km")
    else:
        print("Error fetching data from Overpass API. Please try again later.")

In [105]:
find_grocery_stores(latitude, longitude, radius=2.0)  # within a radius of 2 km

Found 5 grocery stores within 2.0 km, sorted by distance:
Name: Harris Teeter, Coordinates: (38.0499778, -78.5042131), Distance: 0.48 km
Name: Foods of All Nations, Coordinates: (38.0431314, -78.5119486), Distance: 0.60 km
Name: Kroger, Coordinates: (38.0535319, -78.5006204), Distance: 0.97 km
Name: Asian Market, Coordinates: (38.0561993, -78.4966832), Distance: 1.42 km
Name: Super Amanecer, Coordinates: (38.0562741, -78.4966307), Distance: 1.43 km


In [106]:
find_bus_stops(latitude, longitude, radius=0.5)  # within a radius of 0.5 km

Found 8 bus stops within 0.5 km, sorted by distance:
Name: Massie Rd @ JPJ East Lot, Network: Unknown, Coordinates: (38.0447249, -78.5061906), Distance: 0.45 km
Name: Massie Rd @ John Paul Jones Arena, Network: Unknown, Coordinates: (38.0450453, -78.5066476), Distance: 0.39 km
Name: Massie Rd @ JPJ South Lot, Network: Unknown, Coordinates: (38.0458151, -78.5086629), Distance: 0.25 km
Name: Massie Rd @ JPJ West Lot, Network: Unknown, Coordinates: (38.0462614, -78.5093282), Distance: 0.20 km
Name: Massie Rd @ Faulkner Apts (Northbound), Network: Unknown, Coordinates: (38.048134, -78.51205), Distance: 0.26 km
Name: Arlington Blvd @ Millmont St, Network: Unknown, Coordinates: (38.049372, -78.506336), Distance: 0.29 km
Name: Massie Rd @ North Grounds Rec (Northbound), Network: Unknown, Coordinates: (38.0506433, -78.5124984), Distance: 0.42 km
Name: Massie Rd @ Copeley Apts, Network: Unknown, Coordinates: (38.051897, -78.509797), Distance: 0.44 km


In [135]:
find_gyms(latitude, longitude, radius=1.0)  # within a radius of 1 km

Found 9 sports centers within 1.0 km, sorted by distance:
Name: John Paul Jones Arena, Coordinates: (38.0466143, -78.5069309), Distance: 0.25 km
Name: Wilson Collegiate Tennis Camps University of Virginia, Coordinates: (38.0451299, -78.5075791), Distance: 0.35 km
Name: Orangetheory Fitness, Coordinates: (38.0488652, -78.5051546), Distance: 0.36 km
Name: Zoom Training, Coordinates: (38.048768, -78.5049637), Distance: 0.38 km
Name: Zoom Indoor Cycling, Coordinates: (38.0486614, -78.5048193), Distance: 0.39 km
Name: North Grounds Recreation Center, Coordinates: (38.0508858, -78.5135758), Distance: 0.50 km
Name: Quantum Movements Studio, Coordinates: (38.0448655, -78.5169151), Distance: 0.77 km
Name: Salute the Sun Yoga, Coordinates: (38.0447253, -78.516922), Distance: 0.77 km
Name: Stretch Lab, Coordinates: (38.0530763, -78.5009116), Distance: 0.91 km
